### 1. Import packages

In [0]:
#import tensorflow as tf
import numpy as np
import os
import random
import pickle
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.applications import MobileNet
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from keras.layers import LeakyReLU, ReLU
import json
import cv2
import pandas as pd

dirname = "imgs/test/"

In [0]:
# loading of the best parameters
with open('best_params_mobilenet.json') as json_file:
    best_params = json.load(json_file)

# preprocessing of the images and the labels associated
def preprocessing(X, img_rows, img_cols, color_type=3):
    
    X = np.array(X, dtype=np.uint8).reshape(-1, img_rows, img_cols, 3)
    X = X.astype('float32')
    X /= 255
    
    return X

# restoring data
def read_image(path, img_rows, img_cols, color_type=3):
  
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  
  resized_image = cv2.resize(image, (img_cols, img_rows))
  
  return resized_image

# loading of the images
def load_images(img_rows, img_cols, color_type):
    
    images = []
    path = dirname
    img_names = []

    for img in os.listdir(path):

        try:
            image = read_image(os.path.join(path, img), img_rows, img_cols, color_type)     
            images.append(image)
            img_names.append(img)

        except Exception as e:
            pass

    return images, img_names


# definition of the architecture: pre-trained MobileNet + extra layers (fully-connected) optimized
def best_fine_tuning_mobilenet(img_rows, img_cols, color_type, mobilenet_base):

    model = Sequential()
    model.add(mobilenet_base)
    model.add(Flatten())
    model.add(Dense(best_params["n_units_1"]))
    model.add(ReLU())
    model.add(Dropout(best_params["dropout_rate_1"]))
    model.add(Dense(best_params["n_units_2"]))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(Dropout(best_params["dropout_rate_2"]))
    model.add(Dense(10, activation = 'softmax'))

    model.summary()

    model.compile(optimizer = SGD(lr=best_params["lr"], momentum=best_params["momentum"]), loss='categorical_crossentropy', metrics = ["accuracy"])

    return model
    

### 3. Predictions on the test images

In [0]:
img_cols = img_rows = 128
color_type = 3

mobilenet_base = MobileNet(weights=None, include_top=False, input_shape=(img_rows, img_cols, color_type))
model = best_fine_tuning_mobilenet(img_rows, img_cols, color_type, mobilenet_base)

X_test, img_names = load_images(128,128,3)
X_test = preprocessing(X_test,128,128,3)

predicitions_test_list = []

for k in range(1,6):
    print(str(k))
    weights_path = os.path.join('best_config_mobilenet','weights_kfold_' + str(k) + '.h5')

    model.load_weights(weights_path)

    predictions_test = model.predict_proba(X_test, verbose=1)
    predicitions_test_list.append(predictions_test)

predicitions_test_list = np.array(predicitions_test_list)
preditions_mean = np.mean(predicitions_test_list, axis = 0)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Model)   (None, 4, 4, 1024)        3228864   
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 387)               6340995   
_________________________________________________________________
re_lu_1 (ReLU)               (None, 387)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 387)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 107)               41516     
_________________________________________________________________
re_lu_2 (ReLU)               (None, 107)               0         
__________

In [0]:
df = pd.DataFrame(columns=['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])

df['img'] = img_names

CATEGORIES = ["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]
i = 0
for category in CATEGORIES:
    df[category] = preditions_mean[:,i]
    i += 1
df.to_csv("submission_mobilenet.csv",index=False)